# Regression Week 1: Simple Linear Regression

In this notebook we will use data on house sales in King County to predict house prices using simple (one input) linear regression. You will:
* Use graphlab SArray and SFrame functions to compute important summary statistics
* Write a function to compute the Simple Linear Regression weights using the closed form solution
* Write a function to make predictions of the output given the input feature
* Turn the regression around to predict the input given the output
* Compare two different models for predicting house prices

In this notebook you will be provided with some already complete code as well as some code that you should complete yourself in order to answer quiz questions. The code we provide to complte is optional and is there to assist you with solving the problems but feel free to ignore the helper code and write your own.

# Fire up graphlab create

In [6]:
import graphlab

# Load house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [14]:
sales = graphlab.SFrame('kc_house_data.gl/')

# Split data into training and testing

We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [15]:
train_data,test_data = sales.random_split(.8,seed=0)

# Useful SFrame summary functions

In order to make use of the closed form solution as well as take advantage of graphlab's built in functions we will review some important ones. In particular:
* Computing the sum of an SArray
* Computing the arithmetic average (mean) of an SArray
* multiplying SArrays by constants
* multiplying SArrays by other SArrays

In [16]:
# Let's compute the mean of the House Prices in King County in 2 different ways.
prices = sales['price'] # extract the price column of the sales SFrame -- this is now an SArray

# recall that the arithmetic average (the mean) is the sum of the prices divided by the total number of houses:
sum_prices = prices.sum()
num_houses = prices.size() # when prices is an SArray .size() returns its length
avg_price_1 = sum_prices/num_houses
avg_price_2 = prices.mean() # if you just want the average, the .mean() function
print "average price via method 1: " + str(avg_price_1)
print "average price via method 2: " + str(avg_price_2)

average price via method 1: 540088.141905
average price via method 2: 540088.141905


As we see we get the same answer both ways

In [17]:
# if we want to multiply every price by 0.5 it's a simple as:
half_prices = 0.5*prices
# Let's compute the sum of squares of price. We can multiply two SArrays of the same length elementwise also with *
prices_squared = prices*prices
sum_prices_squared = prices_squared.sum() # price_squared is an SArray of the squares and we want to add them up.
print "the sum of price squared is: " + str(sum_prices_squared)

the sum of price squared is: 9.21732513355e+15


Aside: The python notation x.xxe+yy means x.xx \* 10^(yy). e.g 100 = 10^2 = 1*10^2 = 1e2 

# Build a generic simple linear regression function 

Armed with these SArray functions we can use the closed form solution found from lecture to compute the slope and intercept for a simple linear regression on observations stored as SArrays: input_feature, output.

Complete the following function (or write your own) to compute the simple linear regression slope and intercept:

In [61]:
def simple_linear_regression(input_feature, output):
    # compute the sum of input_feature and output
    sum_output=output.sum()
    sum_input=input_feature.sum()
    # compute the product of the output and the input_feature and its sum
    product=output*input_feature
    sumproduct=product.sum()
    # compute the squared value of the input_feature and its sum
    input_square=input_feature*input_feature
    input_squared=input_square.sum()
    # use the formula for the slope
    n=input_feature.size()
    slope=(sumproduct-((sum_output*sum_input)/n))/(input_squared-((sum_input*sum_input)/n))
    # use the formula for the intercept
    intercept= sum_output/n - slope*(sum_input/n)
    return (intercept, slope)

We can test that our function works by passing it something where we know the answer. In particular we can generate a feature and then put the output exactly on a line: output = 1 + 1\*input_feature then we know both our slope and intercept should be 1

In [62]:
import graphlab
test_feature = graphlab.SArray(range(5))
test_output = graphlab.SArray(1 + 1*test_feature)
(test_intercept, test_slope) =  simple_linear_regression(test_feature, test_output)
print "Intercept: " + str(test_intercept)
print "Slope: " + str(test_slope)

Intercept: 1
Slope: 1


Now that we know it works let's build a regression model for predicting price based on sqft_living. Rembember that we train on train_data!

In [63]:
sqft_intercept, sqft_slope = simple_linear_regression(train_data['sqft_living'], train_data['price'])

print "Intercept: " + str(sqft_intercept)
print "Slope: " + str(sqft_slope)

Intercept: -47116.0765749
Slope: 281.958838568


# Predicting Values

Now that we have the model parameters: intercept & slope we can make predictions. Using SArrays it's easy to multiply an SArray by a constant and add a constant value. Complete the following function to return the predicted output given the input_feature, slope and intercept:

In [64]:
def get_regression_predictions(input_feature, intercept, slope):
    # calculate the predicted values:
    predicted_values=input_feature*slope+intercept
    return predicted_values

Now that we can calculate a prediction given the slope and intercept let's make a prediction. Use (or alter) the following to find out the estimated price for a house with 2650 squarefeet according to the squarefeet model we estiamted above.

**Quiz Question: Using your Slope and Intercept from (4), What is the predicted price for a house with 2650 sqft?**

In [65]:
my_house_sqft = 2650
estimated_price = get_regression_predictions(my_house_sqft, sqft_intercept, sqft_slope)
print "The estimated price for a house with %d squarefeet is $%.2f" % (my_house_sqft, estimated_price)

The estimated price for a house with 2650 squarefeet is $700074.85


# Residual Sum of Squares

Now that we have a model and can make predictions let's evaluate our model using Residual Sum of Squares (RSS). Recall that RSS is the sum of the squares of the residuals and the residuals is just a fancy word for the difference between the predicted output and the true output. 

Complete the following (or write your own) function to compute the RSS of a simple linear regression model given the input_feature, output, intercept and slope:

In [66]:
def get_residual_sum_of_squares(input_feature, output, intercept, slope):
    # First get the predictions
    final_output=input_feature*slope+intercept
    print final_output
    print output
    # then compute the residuals (since we are squaring it doesn't matter which order you subtract)
    errors=output-final_output
    
    # square the residuals and add them up
    rss=(errors*errors)
    
    RSS=rss.sum()
    return(RSS)

Let's test our get_residual_sum_of_squares function by applying it to the test model where the data lie exactly on a line. Since they lie exactly on a line the residual sum of squares should be zero!

In [67]:
print get_residual_sum_of_squares(test_feature, test_output, test_intercept, test_slope) # should be 0.0

[1L, 2L, 3L, 4L, 5L]
[1L, 2L, 3L, 4L, 5L]
0


Now use your function to calculate the RSS on training data from the squarefeet model calculated above.

**Quiz Question: According to this function and the slope and intercept from the squarefeet model What is the RSS for the simple linear regression using squarefeet to predict prices on TRAINING data?**

In [68]:
rss_prices_on_sqft = get_residual_sum_of_squares(train_data['sqft_living'], train_data['price'], sqft_intercept, sqft_slope)
print 'The RSS of predicting Prices based on Square Feet is : ' + str(rss_prices_on_sqft)

[285595.35293494293, 677518.1385440424, 169992.22912218698, 505523.2470177469, 426574.77221879165, 1481100.82846198, 436443.33156866103, 251760.29230681923, 454770.65607556136, 485786.12831800804, 956657.3887260627, 279956.176163589, 339167.53226280544, 463229.4212325923, 485786.12831800804, 404018.0651333758, 291234.52970629686, 305332.47163468174, 409657.2419047298, 812858.381056537, 592930.4869737331, 254579.8806924962, 643683.0779159187, 643683.0779159187, 347626.29741983634, 381461.35804796004, 677518.1385440424, 288414.9413206199, 609848.017287795, 533719.1308745167, 601389.2521307641, 420935.59544743766, 618306.7824448259, 296873.70647765085, 677518.1385440424, 966525.9480759321, 395559.2999763449, 313791.2367917127, 843873.8532989838, 232023.17360708042, 598569.663745087, 305332.47163468174, 729116.6060019309, 288414.9413206199, 841054.2649133068, 350445.8858055133, 511162.42378910084, 722631.5527148739, 750827.4365716437, 635224.3127588877, 869250.1487700766, 474507.7747753002

# Predict the squarefeet given price

What if we want to predict the squarefoot given the price? Since we have an equation y = a + b\*x we can solve the function for x. So that if we have the intercept (a) and the slope (b) and the price (y) we can solve for the estimated squarefeet (x).

Complete the following function to compute the inverse regression estimate, i.e. predict the input_feature given the output.

In [69]:
def inverse_regression_predictions(output, intercept, slope):
    # solve output = intercept + slope*input_feature for input_feature. Use this equation to compute the inverse predictions:
    estimated_feature=(output-intercept)/slope
    return estimated_feature

Now that we have a function to compute the squarefeet given the price from our simple regression model let's see how big we might expect a house that costs $800,000 to be.

**Quiz Question: According to this function and the regression slope and intercept from (3) what is the estimated square-feet for a house costing $800,000?**

In [70]:
my_house_price = 800000
estimated_squarefeet = inverse_regression_predictions(my_house_price, sqft_intercept, sqft_slope)
print "The estimated squarefeet for a house worth $%.2f is %d" % (my_house_price, estimated_squarefeet)

The estimated squarefeet for a house worth $800000.00 is 3004


# New Model: estimate prices from bedrooms

We have made one model for predicting house prices using squarefeet, but there are many other features in the sales SFrame. 
Use your simple linear regression function to estimate the regression parameters from predicting Prices based on number of bedrooms. Use the training data!

In [71]:
# Estimate the slope and intercept for predicting 'price' based on 'bedrooms'
bed_intercept, bed_slope = simple_linear_regression(train_data['bedrooms'], train_data['price'])

print "Intercept: " + str(bed_intercept)
print "Slope: " + str(bed_slope)


Intercept: 109473.180469
Slope: 127588.952175


# Test your Linear Regression Algorithm

Now we have two models for predicting the price of a house. How do we know which one is better? Calculate the RSS on the TEST data (remember this data wasn't involved in learning the model). Compute the RSS from predicting prices using bedrooms and from predicting prices using squarefeet.

**Quiz Question: Which model (square feet or bedrooms) has lowest RSS on TEST data? Think about why this might be the case.**

In [72]:
# Compute RSS when using bedrooms on TEST data:
rss_prices_on_bed = get_residual_sum_of_squares(test_data['sqft_living'], test_data['price'], bed_intercept, bed_slope)
print 'The RSS of predicting Prices based on Square Feet is : ' + str(rss_prices_on_bed)

[182561674.79012406, 376496882.0954914, 218286581.39900753, 296115842.2255036, 139181431.05076557, 334392527.8778787, 538534851.3572128, 287184615.5732828, 160871552.92044482, 350979091.6605746, 228493697.57297423, 242528482.31217843, 252735598.48614514, 250183819.44265348, 259115046.09487432, 412221788.70437485, 371393324.008508, 149388547.22473228, 359910318.3127955, 135353762.48552808, 325461301.2256579, 159595663.398699, 262942714.66011184, 257839156.5731285, 342047865.00835377, 199148238.57281995, 495154607.6178543, 143009099.6160031, 213183023.31202418, 247632040.3991618, 116215419.65934052, 316530074.57343704, 311426516.48645365, 108560082.5288655, 223390139.48599088, 192768790.96409076, 219562470.92075336, 479843933.35690427, 465809148.61770004, 231045476.6164659, 268046272.7470952, 97077076.83315295, 217010691.8772617, 331840748.83438706, 109835972.05061133, 488775160.0091251, 446670805.7915125, 308874737.442962, 275701609.8775702, 251459708.9643993, 260390935.61662015, 280805

In [73]:
# Compute RSS when using squarefeet on TEST data:
rss_prices_on_sqft = get_residual_sum_of_squares(test_data['sqft_living'], test_data['price'], sqft_intercept, sqft_slope)
print 'The RSS of predicting Prices based on Square Feet is : ' + str(rss_prices_on_sqft)

[356085.0625768673, 784662.4971997673, 435033.53737582255, 607028.428902118, 260219.0574638502, 691616.0804724272, 1142750.222180743, 587291.3102023791, 308152.0600203587, 728270.7294862279, 457590.2444612383, 488605.716703685, 511162.42378910084, 505523.2470177469, 525260.3657174857, 863610.9719987226, 773384.1436570594, 282775.76454926596, 748007.8481859666, 251760.29230681923, 671878.9617726883, 305332.47163468174, 533719.1308745167, 522440.7773318087, 708533.610786489, 392739.71159066795, 1046884.217067726, 268677.8226208811, 423755.1838331146, 499884.070246393, 209466.46652166464, 652141.8430729496, 640863.4895302417, 192548.9362076028, 446311.89091853047, 378641.76966228307, 437853.1257614995, 1013049.1564396021, 982033.6841971555, 463229.4212325923, 544997.4844172245, 167172.64073651002, 432213.9489901456, 685976.9037010733, 195368.52459327976, 1032786.2751393411, 939739.8584120009, 635224.3127588877, 561915.0147312863, 508342.8354034239, 528079.9541031627, 573193.3682739943, 11